In [1]:
import json
import os
import re  # Importar expresiones regulares para eliminar menciones en tweets

# Obtener el directorio actual donde están los archivos
directorio_actual = os.getcwd()

# Definir rutas de los archivos
archivo_sentimientos = os.path.join(directorio_actual, "Sentimientos.txt")
archivo_tweets = os.path.join(directorio_actual, "Tweets.txt")
archivo_resultados = os.path.join(
    directorio_actual, "Resultados_Sentimientos_Desconocidos.txt"
)

# Cargar el diccionario de sentimientos desde el archivo Sentimientos.txt
with open(archivo_sentimientos, "r", encoding="utf-8") as sentimientos:
    valores = {}  # Diccionario que almacena {palabra: sentimiento}
    for linea in sentimientos:
        termino, valor = linea.strip().split(
            "\t"
        )  # Separar palabra y valor por tabulación
        valores[termino] = int(
            valor
        )  # Guardar en el diccionario con conversión a entero


def limpiar_tweet(tweet):
    """Eliminar menciones, hashtags, enlaces, emojis, caracteres no latinos, 'RT', signos de puntuación, fechas, números y códigos HTML.

    Args:
        tweet (str): Texto original del tweet

    Returns:
        str or None: Texto limpio sin elementos no deseados, o None si queda vacío
    """
    # Eliminar "RT" en cualquier posición
    tweet = re.sub(r"\bRT\b", "", tweet)

    # Eliminar menciones (@usuario), hashtags (#hashtag) y enlaces (http:// o https://)
    tweet = re.sub(r"(@\w+|#\w+|https?://\S+|www\.\S+)", "", tweet)

    # Eliminar emojis
    tweet = re.sub(
        r"[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F"
        r"\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F"
        r"\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+",
        "",
        tweet,
    )

    # Eliminar caracteres no latinos (chino, japonés, griego, etc.)
    tweet = re.sub(r"[^\x00-\x7F]+", "", tweet)

    # Eliminar signos de puntuación y caracteres especiales como '|'
    tweet = re.sub(r"[.,!?;:(){}\[\]\"'_\-@|]", "", tweet)

    # Eliminar fechas en formatos comunes (DD/MM/YY, YYYY-MM-DD, etc.)
    tweet = re.sub(r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b", "", tweet)

    # Eliminar números sueltos
    tweet = re.sub(r"\b\d+\b", "", tweet)

    # Eliminar códigos HTML (&amp, &lt3, etc.)
    tweet = re.sub(r"&\w+|\bhttp/\b|\bcomandroidinternalosZygoteInit\w+\b", "", tweet)

    # Eliminar espacios extra y devolver None si el tweet queda vacío
    tweet = tweet.strip()
    return tweet if tweet else None


def calcular_sentimiento(tweet, valores):
    """Calcular el sentimiento sumando los valores asociados a palabras conocidas.

    Args:
        tweet (str): Texto del tweet procesado
        valores (dict): Diccionario con palabras y sus valores de sentimiento

    Returns:
        int: Sentimiento total del tweet
    """
    palabras = tweet.split()  # Dividir el tweet en palabras
    return sum(
        valores.get(palabra, 0) for palabra in palabras
    )  # Sumar los valores de palabras conocidas


def obtener_palabras_desconocidas(tweet, valores, sentimiento_tweet):
    """Obtener palabras que no tienen un sentimiento asociado en el diccionario y asignarles un valor basado en el sentimiento del tweet.

    Args:
        tweet (str): Texto del tweet procesado
        valores (dict): Diccionario con palabras y sus valores de sentimiento
        sentimiento_tweet (int): Sentimiento total del tweet

    Returns:
        list: Lista de palabras desconocidas (sin valor en el diccionario)
    """
    palabras = tweet.split()
    palabras_desconocidas = [
        palabra for palabra in palabras if palabra.lower() not in valores
    ]  # Filtrar palabras no registradas

    # Opción 1: Asignar el sentimiento total del tweet a las palabras desconocidas
    sentimiento_desconocidas = sentimiento_tweet

    # Opción 2: Asignar la media del sentimiento del tweet a las palabras desconocidas
    # sentimiento_desconocidas = sentimiento_tweet / len(palabras)

    return palabras_desconocidas, sentimiento_desconocidas


def procesar_tweets():
    """Leer los tweets, calcular sus sentimientos y asignar valores a palabras desconocidas."""
    try:
        # Abrir archivos: tweets en modo lectura y resultados en modo escritura
        with open(archivo_tweets, "r", encoding="utf-8") as f, open(
            archivo_resultados, "w", encoding="utf-8"
        ) as out:
            for line in f:  # Leer línea por línea el archivo de tweets
                try:
                    tweet_json = json.loads(line)  # Cargar cada línea como JSON
                    texto_tweet = tweet_json.get(
                        "text", ""
                    ).strip()  # Extraer el texto del tweet

                    texto_tweet = limpiar_tweet(
                        texto_tweet
                    )  # Limpiar menciones (@usuario)

                    if texto_tweet:  # Solo procesar si el tweet tiene contenido
                        sentimiento = calcular_sentimiento(
                            texto_tweet, valores
                        )  # Calcular sentimiento del tweet
                        palabras_desconocidas, sentimiento_desconocidas = (
                            obtener_palabras_desconocidas(
                                texto_tweet, valores, sentimiento
                            )
                        )  # Buscar palabras sin valor

                        for palabra in palabras_desconocidas:
                            resultado = f"'{palabra}': {sentimiento_desconocidas}\n"
                            print(resultado, end="")  # Mostrar en pantalla
                            out.write(resultado)  # Guardar en archivo

                except json.JSONDecodeError:
                    continue  # Ignorar líneas que no sean JSON válido

    except FileNotFoundError:
        print(f"No se encontró el archivo '{archivo_tweets}'.")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")


# Ejecutar el procesamiento de tweets
procesar_tweets()
print(f"Resultados guardados en: {archivo_resultados}")

'I': 0
'WHALE': 0
'SLAP': 0
'YOUR': 0
'FIN': 0
'AND': 0
'TELL': 0
'YOU': 0
'ONE': 0
'LAST': 0
'TIME': 0
'GO': 0
'AWHALE': 0
'Metin': 0
'entrk': 0
'Twitterda': 0
'MUHTEEM': 0
'L': 0
'SEN': 0
'BEN': 0
'MZK': 0
'I': 0
'SO': 0
'MUCH': 0
'IS': 0
'GOING': 0
'TO': 0
'BE': 0
'SO': 0
'que': 0
'hdp': 0
'maicon': 0
'lo': 0
'que': 0
'le': 0
'hizo': 0
'a': 0
'david': 0
'luiz': 0
'jajajajajajajajajajajigual': 0
'se': 0
'jodio': 0
'la': 0
'carrera': 0
'tudo': 0
'o': 0
'que': 0
'eu': 0
'quero': 0
'um': 0
'amor': 0
'sincero': 0
'Si': 0
'Nathalie': 0
'sait': 0
'pertinemment': 0
'que': 0
'son': 0
'couple': 0
'ne': 0
'va': 0
'pas': 0
'durer': 0
'alors': 0
'pourquoi': 0
'accepter': 0
'la': 0
'demande': 0
'en': 0
'mariage': 0
'de': 0
'Vivian': 0
'Junior': 0
'va': 0
'perdiendo': 0
'tudo': 0
'o': 0
'que': 0
'eu': 0
'quero': 0
'um': 0
'amor': 0
'sincero': 0
'tudo': 0
'o': 0
'que': 0
'eu': 0
'quero': 0
'um': 0
'amor': 0
'sincero': 0
'TWTTERI': 0
'AKRABALAR': 0
'KEFEDCEK': 0
'DYE': 0
'OK': 0
'KORKUYORUM': 0
'Met